208. Implement Trie (Prefix Tree)
1233. Remove Sub-Folders from the Filesystem
1032. Stream of Characters
211. Add and Search Word - Data structure design
676. Implement Magic Dictionary
677. Map Sum Pairs
745. Prefix and Suffix Search
425. Word Squares

In [26]:
class Trie:
    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root = {}
        self.words = {} # 677. Map Sum Pairs, each word gives a val
        self.end = '#'
 
    def insert(self, word):
        """
        Inserts a word into the trie.
        :type word: str
        :rtype: void
        """
        
        curNode = self.root
        for c in word:
            if not c in curNode:
                curNode[c] = {}
            curNode = curNode[c]
        curNode[self.end] = True
        
    def insertWithVal(self, key, val): # 677. Map Sum Pairs，给单词一个val
        """
        :type key: str
        :type val: int
        :rtype: None
        """
        cur = self.root
        if key in self.words:
            self.words[key] = val
            return
        for i in key:
            if i not in cur:
                cur[i] = dict()
            cur = cur[i]
        cur[self.end] = True
        self.words[key] = val
        return    
 
    def search(self, word):
        """
        Returns if the word is in the trie.
        :type word: str
        :rtype: bool
        """
        curNode = self.root
        for c in word:
            if not c in curNode:
                return False
            curNode = curNode[c]

        # Doesn't end here
        if not self.end in curNode:
            return False
        
        return True
    
    def fuzzySearch(self, word):# 211 Add and Search Word - Data structure design，含有通配符的搜索
        """
        Returns if the word is in the data structure. A word could contain the dot character '.' to represent any one letter.
        :type word: str
        :rtype: bool
        """
        def searchWord(word,trie):
            cur = trie
            for c in range(len(word)):
                if word[c]!='.' and word[c] not in cur:
                    return
                elif word[c] in cur:
                    cur = cur[word[c]]
                else:
                    for key in cur.keys():
                        if key!=self.end:
                            searchWord(word[c+1:],cur[key])
                    return
            if self.end in cur:
                self.find = True
        self.find = False
        searchWord(word,self.root)
        return self.find
    
    def magicSearch(self, word):# 676. Implement Magic Dictionary，当替换一个char能找到时返回True
        """
        Returns if there is any word in the trie that equals to the given word after modifying exactly one character
        :type word: str
        :rtype: bool
        """
        def wordSearch(word,trie,option):
            for i in range(len(word)):
                if option:
                    for key in trie:
                        if key!='#':
                            if key == word[i]:
                                wordSearch(word[i+1:],trie[key],1)
                            else:
                                wordSearch(word[i+1:],trie[key],0)  
                    return
                else:
                    if word[i] in trie:# return要放在外面，担心不会进入if语句
                        wordSearch(word[i+1:],trie[word[i]],option)
                    return
            if '#' in trie and option==0:
                self.res = True
        self.res = False
        wordSearch(word,self.root,1)
        return self.res
    
    def sum_prefix(self, prefix): #677. Map Sum Pairs, sum values for word with given prefix
        """
        :type prefix: str
        :rtype: int
        """
        cur = self.root
        for i in prefix:
            if i not in cur:
                return 0
            else:
                cur = cur[i]
                
        def list_words(cur):#将cur字典下的所有单词全部列出来
            my_list = []
            for k,v in cur.items():
                if k != self.end:
                    for el in list_words(v):#下一层的单词都加上k
                        my_list.append(k+el)
                else:
                    my_list.append('')
            return my_list
            
        res = 0
        wordlist = list_words(cur)
        for i in wordlist:
            res += self.words[prefix+i]
        return res
 
    def startsWith(self, prefix):
        """
        Returns if there is any word in the trie that starts with the given prefix.
        :type prefix: str
        :rtype: bool
        """
        curNode = self.root
        for c in prefix:
            if not c in curNode:
                return False
            curNode = curNode[c]
        
        return True
    
    def list_words(self,level):# return all the words, including possible ""
        my_list = []
        for k,v in level.items():
            if k != self.end:
                for el in self.list_words(v):
                    my_list.append(k+el)
            else:
                my_list.append('')
        return my_list

In [25]:
obj = Trie()
words = ["mobile","mouse","moneypot","monitor","mousepad",""]
for word in words:
    obj.insertWith(word)
obj.search('mobile')

True

In [35]:
# 677. Map Sum Pairs, sum values for word with given prefix
sol = Trie()
sol.insertWithVal("apple", 3)
sol.insertWithVal("app", 2)
sol.insertWithVal("ap", 4)
sol.insertWithVal("a", 5)
sol.sum_prefix('a')

14

In [133]:
# 648. Replace Words,对sentence里的单词从dict里寻找前缀
def replaceWords(dict, sentence):
    """
    :type dict: List[str]
    :type sentence: str
    :rtype: str
    """
    tr = Trie()
    for word in dict:
        tr.insert(word)
    words = sentence.split(' ')
    def replace(word,tr):
        cur = tr.root
        for i in range(len(word)):
            if tr.end in cur:
                return word[:i]
            if word[i] not in cur:
                return word
            cur = cur[word[i]]
        return word

    for i in range(len(words)):
        words[i] = replace(words[i],tr)
    return " ".join(words)
replaceWords(["cat", "bat", "rat"],"the cattle was rattled by the battery")

'the cat was rat by the bat'

In [19]:
# 212. Word Search II，给定board和单词表，找出出现过的单词路径
# 先构建trie，然后沿着trie一层层比对
def findWords(board, words):
    """
    :type board: List[List[str]]
    :type words: List[str]
    :rtype: List[str]
    """
    trie = dict()
    end = '#'
    if not board:
        return []
    m,n = len(board),len(board[0])

    for word in words:# build the trie
        cur = trie
        for i in word:
            if i not in cur:
                cur[i] = dict()
            cur = cur[i]
        cur[end] = True

    def searchPrefix(keys,board):# 找出所有可能的prefix
        pos = dict()
        for i in range(m):
            for j in range(n):
                if board[i][j] in keys:
                    pos[board[i][j]] = pos.get(board[i][j],[]) + [(i,j)]
        return pos

    def searchWord(pre,pos,trie,visited,path,res):
        """
        pre: the current prefix
        pos: the position of pre
        trie: prefix tree which has the same root as pre
        visited: visited node to avoid conflicting with itself
        """
        if end in trie[pre]:
            res.add(path)
        for dx,dy in [(-1,0),(1,0),(0,1),(0,-1)]:
            xnew, ynew = pos[0]+dx, pos[1]+dy
            if 0<=xnew<m and 0<=ynew<n and board[xnew][ynew] in trie[pre] and (xnew,ynew) not in visited:
                searchWord(board[xnew][ynew],(xnew,ynew),trie[pre],visited|{(xnew,ynew)},path+board[xnew][ynew],res)

    poses = searchPrefix(trie.keys(),board)
    res = set()
    for pre in poses:
        for pos in poses[pre]:
            searchWord(pre,pos,trie,{pos},pre,res)
    return list(res)

findWords([["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],["oath","pea","eat","rain",'v','oathk'])

['v', 'oath', 'eat', 'oathk']

In [26]:
# 472. Concatenated Words,返回words中能由别的字符串构成的字符串
# 将字符串构成trie，对每一个单词遍历，如果能断开，则断开尝试，或者如果可以往下走就往下走，需要判断是否为原word
def findAllConcatenatedWordsInADict(words):
    """
    :type words: List[str]
    :rtype: List[str]
    """
    trie = dict()
    end = '#'

    def insert(word):
        cur = trie
        for i in word:
            if i not in cur:
                cur[i] = dict()
            cur = cur[i]
        cur[end] = True

    for word in words:
        if word:
            insert(word)

    def dfs(node,word,cur,space):
        # space=True说明已经断开过了
        if cur == len(word):
            return end in node and space
        if end in node:#如果碰到可以断开的，就断开
            if dfs(trie,word,cur,True):#如果断开能成功，则成功
                return True
        if word[cur] in node:
            return dfs(node[word[cur]],word,cur+1,space)
#         else:# 这里其实不需要，因为最终一定会走向cur
#             return False
        
    res = []
    for word in words:
        if dfs(trie,word,0,False):
            res.append(word)
    return res

def findAllConcatenatedWordsInADict(words):
    """
    :type words: List[str]
    :rtype: List[str]
    """
    trie = dict()
    end = ['#']

    words.sort(key=len)#前面的一定不需要后面的

    def insert(word):
        cur = trie
        for i in word:
            if i not in cur:
                cur[i] = dict()
            cur = cur[i]
        cur[end[0]] = True

    def dfs(word):
        cur = trie
        for i,c in enumerate(word):
            if c not in cur:#如果不在里面，则返回False
                return False
            cur = cur[c]
            if end[0] in cur and dfs(word[i+1:]):#在里面且分段，如果是最后一段，则会直接走完loop，确认end是否在
                return True
        return end[0] in cur

    res = []
    for word in words:
        if len(word)==0:
            continue
        if dfs(word):#如果已经满足，则不需要插入到trie
            res.append(word)
        else:
            insert(word)
    return res
    
findAllConcatenatedWordsInADict(["cat","cats","catsdogcats","dog","dogcatsdog","hippopotamuses","rat","ratcatdogcat"])

['dogcatsdog', 'catsdogcats', 'ratcatdogcat']

In [4]:
# 1268. Search Suggestions System，给定searchword返回每个前缀对应的单词
def suggestedProducts(products, searchWord):
    trie = {}
    end = '#'
    for word in products:
        curword = trie
        for i in word:
            if i not in curword:
                curword[i] = dict()
            curword = curword[i]
        if end not in curword:
            curword[end] = 0
        curword[end]+=1

    
    def searchK(curword,target):
        res = []
        keys = sorted(curword.keys())
        for key in keys:
            if key=='#':
                res+= ["" * curword[end]]
                if len(res)>target:
                    break
            else:
                temp = searchK(curword[key],target-len(res))
                for i in temp:
                    res.append(key+i)
                if len(res)>target:
                    break
        return res[:target]
                        
    curword = trie
    ans = []
    for i in range(len(searchWord)):
        if searchWord[i] not in curword:
            ans = ans+[[] for _ in range(len(searchWord)-i)]
            break
        curword = curword[searchWord[i]]
        res = [searchWord[:i+1]+t for t in searchK(curword,3)]
        ans.append(res)
    return ans
print(suggestedProducts(products = ["mobile","mouse","moneypot","monitor","mousepad"], searchWord = "mouse"))

# 排序以后用二分法，python可以用startswith,可以用string比较顺序
def suggestedProducts(A, word):
    A.sort()
    res, prefix, i = [], '', 0
    for c in word:
        prefix += c
        i = bisect.bisect_left(A, prefix, i)
        res.append([w for w in A[i:i + 3] if w.startswith(prefix)])
    return res

[['mobile', 'moneypot', 'monitor'], ['mobile', 'moneypot', 'monitor'], ['mouse', 'mousepad'], ['mouse', 'mousepad'], ['mouse', 'mousepad']]


In [79]:
# 642. Design Search Autocomplete System
class AutocompleteSystem(object):

    def __init__(self, sentences, times):
        """
        :type sentences: List[str]
        :type times: List[int]
        """
        self.trie = dict()
        self.end = "##"
        for words,time in zip(sentences,times):
            self.insert(words,time)
        self.current = self.trie
        self.record = ""
        
    def insert(self,words,time):
        cur = self.trie
        for c in words:
            if c not in cur:
                cur[c] = dict()
            cur = cur[c]
        cur[self.end] = cur.get(self.end,0)+time
        return 
    
    def list_words(self,cur):#list words under current directory
        words = []
        for key,val in cur.items():
            if key!=self.end:
                for word,value in self.list_words(cur[key]):
                    words.append([key+word,value])
            else:
                words.append(["",val])
        return words

    def input(self, c):
        """
        :type c: str
        :rtype: List[str]
        """
        if c=='#':#一个搜索结束后，将搜索记录添加，并重置trie与搜索记录
            self.insert(self.record,1)
            self.current = self.trie
            self.record = ""
            return []
        self.record+=c
        if c not in self.current:#如果没找到，也要将搜索往前走
            self.current = dict()
            return []
        res = sorted(self.list_words(self.current[c]),key=lambda x:(-x[1],x[0]))
        self.current = self.current[c]
        if len(res)<=3:
            return [self.record+i[0] for i in res]
        else:
            return [self.record+i[0] for i in res[:3]]

In [26]:
# 1233. Remove Sub-Folders from the Filesystem,移除所有的subfolder
def removeSubfolders(folder):
    """
    :type folder: List[str]
    :rtype: List[str]
    """
    folder.sort()#排序
    trie = dict()
    res = []
    end = '#'
    for word in folder:
        words = word.split('/')[1:]
        cur = trie
        flag = False
        for i in words:
            if i in cur:
                cur = cur[i]
                if '#' in cur:
                    flag = True
                    break
            else:
                cur[i] = dict()
                cur = cur[i]
        if not flag:
            cur[end] = True
            res.append(word)
    return res
removeSubfolders( ["/c/d","/c/d/e"])

['/c/d']

In [79]:
# 820. Short Encoding of Words
def minimumLengthEncoding(words):
    """
    :type words: List[str]
    :rtype: int
    """
    trie = dict()
    end = '#'

    for word in words:
        cur = trie
        for i in word[::-1]:
            if i not in cur:
                cur[i] = dict()
            cur = cur[i]
        cur[end] = len(word)

    res = 0
    level = [trie[i] for i in trie]#这一部分容易错
    while level:
        newlevel = []
        for node in level:
            if len(node)>=2:
                for key in node:
                    if key!=end:
                        newlevel.append(node[key])
            else:
                if end in node:
                    res+=node[end]+1
                else:
                    node = node[list(node.keys())[0]]
                    newlevel.append(node) 
        level = newlevel
    return res

def minimumLengthEncoding(words):
    root = dict()
    leaves = []
    for word in set(words):
        cur = root
        for i in word[::-1]:
            cur[i] = cur = cur.get(i, dict())
        leaves.append((cur, len(word) + 1))
    return sum(depth for node, depth in leaves if len(node) == 0)

def minimumLengthEncoding(words):#把所有的后缀部分都丢掉
    s = set(words)
    for w in words:
        for i in range(1, len(w)):
            s.discard(w[i:])
    return sum(len(w) + 1 for w in s)
minimumLengthEncoding(words =["time", "me", "bell",'a','ebe','dew','afa','efa','fdaf','ega'])

35

In [1]:
# 1065. Index Pairs of a String，给定一系列words，在text中找对应的单词substring
def indexPairs(text, words):
    """
    :type text: str
    :type words: List[str]
    :rtype: List[List[int]]
    """
    trie = dict()
    end = '#'
    res = []

    for word in words:
        cur = trie
        for i in word:
            if i not in cur:
                cur[i] = dict()
            cur = cur[i]
        cur[end] = True
    # print(trie)

    for i in range(len(text)):
        j = i
        cur = trie
        while j<len(text) and text[j] in cur:
            cur = cur[text[j]]
            if end in cur:
                res.append([i,j])
            j = j+1
    return res

In [43]:
# 745. Prefix and Suffix Search，给定prefix和suffix在一堆words中寻找最近的单词

# 对每一个单词，直接存储所有可能的prefix,suffix，适合大量查询+短单词
class WordFilter(object):

    def __init__(self, words):
        self.hashtable = {}
        for index, word in enumerate(words):
            prefix = ''
            for char in [''] + list(word):
                prefix += char
                suffix = ''
                for char in [''] + list(word[::-1]):
                    suffix += char
                    self.hashtable[prefix + '.' + suffix[::-1]] = index

    def f(self, prefix, suffix):
        return self.hashtable.get(prefix + '.' + suffix, -1)
    
# 存储每个的suffix和prefix，然后在找到的两个列表里寻找重复元素，为避免重复，倒着存储，两个列表为单调递减，用two pointers查找
class WordFilter(object):

    def __init__(self, words):
        """
        :type words: List[str]
        """
        self.prefix = dict()
        self.suffix = dict()
        self.visited = set()
        for i in range(len(words)-1,-1,-1):
            word = words[i]
            if word in self.visited:
                continue
            pre = ""
            self.prefix[pre] = self.prefix.get(pre,[])+[i]
            for c in word:
                pre += c
                self.prefix[pre] = self.prefix.get(pre,[])+[i]
            suf = ""
            self.suffix[suf] = self.suffix.get(suf,[])+[i]
            for c in word[::-1]:
                suf = c+suf
                self.suffix[suf] = self.suffix.get(suf,[])+[i]
            self.visited.add(word)

    def f(self, prefix, suffix):
        """
        :type prefix: str
        :type suffix: str
        :rtype: int
        """
        if prefix not in self.prefix or suffix not in self.suffix:
            return -1
        i,j = 0,0
        while i<len(self.prefix) and j<len(self.suffix):
            if self.prefix[prefix][i]==self.suffix[suffix][j]:
                return self.prefix[prefix][i]
            elif self.prefix[prefix][i]>self.suffix[suffix][j]:
                i+=1
            else:
                j+=1
        return -1

In [81]:
# 336. Palindrome Pairs，给定一堆单词，如果两个单词拼接一起构成回文，返回index

def palindromePairs(words):
    """
    :type words: List[str]
    :rtype: List[List[int]]
    """
    trie = dict()
    end = '#'
    pars = dict()#对于每个单词，其前面的已经构成palindrome的长度
    for word in words:#这一部分复杂度为nw
        maximum = []
        for i in range(len(word)):
            if word[:i+1] == word[:i+1][::-1]:
                maximum.append(i+1)
        pars[word] = maximum

    for index,word in enumerate(words):
        cur = trie
        for c in word:
            if c not in cur:
                cur[c] = dict()
            cur = cur[c]
        cur[end] = index
    print(trie,pars)

    def list_words(level):# return all the words, including possible ""
        my_list = []
        for k,v in level.items():
            if k != end:
                for el,index in list_words(v):
                    my_list.append((k+el,index))
            else:
                my_list.append(("",v))
        return my_list

    def getPalin(cur):#get palindromes for a trie structure
        curwords = list_words(cur)
        curres = []
        for word,index in curwords:
            if word == word[::-1]:
                curres.append(index)
        return curres

    res = []
    for index,word in enumerate(words):
        curlen = 0
        cur = trie
        flag = False
        for i,c in enumerate(word[::-1]):
            if len(word)-i in pars[word] and end in cur and cur[end]!=index:#如果已经有一部分满足了回文，不需要完整的单词就可以
                res.append([cur[end],index])
            if c in cur:
                cur=cur[c]
            else:
                flag = True
                break
        if not flag:#如果走完了，可能剩下的部分自身就是回文
            for t in getPalin(cur):
                if t!=index:
                    res.append([t,index])
    return res

# 对word每一个可能的部分做反转然后去寻找是否存在对应项
def palindromePairs(words):# 复杂度为nw^2    
    def is_palindrome(check):
        return check == check[::-1]

    words = {word: i for i, word in enumerate(words)}
    print(words)
    valid_pals = []
    for word, k in words.items():
        n = len(word)
        for j in range(n+1):
            pref = word[:j]
            suf = word[j:]
            if is_palindrome(pref):#如果前面这一部分(从0到全部长度）是回文，那么找到后面这一部分应该对应的补足
                back = suf[::-1]
                if back != word and back in words:
                    valid_pals.append([words[back],  k])
            if j != n and is_palindrome(suf):#如果后面这一部分是回文，避免后一部分为空，避免重复（虽然和前面相反，但是会和后面撞上）
                back = pref[::-1]
                if back != word and back in words:
                    valid_pals.append([k, words[back]])
    return valid_pals
palindromePairs(["abcd","dcba","lls","s","sssll"])

{'abcd': 0, 'dcba': 1, 'lls': 2, 's': 3, 'sssll': 4}


[[1, 0], [0, 1], [3, 2], [2, 4]]

In [1]:
# 425. Word Squares,单词构成方块的所有种类,对每个单词进行尝试，尝试之后确定前缀，然后根据前缀寻找可能的下一个
def wordSquares(words):
    """
    :type words: List[str]
    :rtype: List[List[str]]
    """
    n = len(words[0])
    if n==1:
        return [[i] for i in words]

    trie = dict()
    end = ['#']
    for word in words:
        cur = trie
        for c in word:
            if c not in cur:
                cur[c] = dict()
            cur = cur[c]
        cur[end[0]] = word

    def list_words(cur):#找出前缀下的所有单词
        cur_list = []
        for key,value in cur.items():
            if key!=end[0]:
                for v in list_words(value):
                    cur_list.append(key+v)
            else:
                cur_list.append("")
        return cur_list

    def dfs(size,path,res):
        if size==0:
            for i,word in enumerate(words):
                dfs(size+1,path+[word],res)
        elif size==len(path[0]):
            res.append(path)
            return
        else:
            prefix = "".join([word[size] for word in path])#prefix由之前的path决定
            cur = trie
            for c in prefix:
                if c not in cur:
                    return
                cur = cur[c]
            possible_words = [prefix+i for i in list_words(cur)]
            for word in possible_words:#可以使用重复的单词
                dfs(size+1,path+[word],res)

    res = []
    dfs(0,[],res)
    return res
wordSquares(["abat","baba","atan","atal"])

[['baba', 'abat', 'baba', 'atan'], ['baba', 'abat', 'baba', 'atal']]

In [2]:
# 面试题 17.25. 单词矩阵
import collections
def maxRectangle(words):
    """
    :type words: List[str]
    :rtype: List[str]
    """
    trie = dict()
    end = '#'
    for word in words:
        cur = trie
        for c in word:
            if c not in cur:
                cur[c] = dict()
            cur = cur[c]
        cur[end] = True

    hashtable = collections.defaultdict(set)
    for word in words:
        hashtable[len(word)].add(word)

    def dfs(arr,path,area,ans):#每一列的初始trie,path,area,ans
        count = 0
        for i in arr:#数一数有多少列已经组成了单词
            if '#' in i:
                count+=1
        if count == len(arr) and area>res[0]:
            res[0] = area
            ans = path
        for word in hashtable[len(arr)]:#对该长度的单词一个个搜索，该单词是否满足每一个trie都能找到下家
            arr2 = []
            for index,c in enumerate(word):
                if c not in arr[index]:
                    break
                arr2.append(arr[index][c])
            else:#如果该单词可行，继续往下走
                ans = dfs(arr2,path+[word],area+len(arr),ans)
        return ans

    res = [0]
    ans = []
    keys = sorted(hashtable.keys(),reverse=True)
    for key in keys:#当前长度为key
        if key*keys[0]<=res[0]:#如果当前长度不可能更大，就退出
            break
        ans = dfs([trie for _ in range(key)],[],0,ans)
    return ans
maxRectangle(["this", "real", "hard", "trh", "hea", "iar", "sld"])

['this', 'real', 'hard']

In [15]:
# 牛客网，https://www.nowcoder.com/practice/fc05f68c5f47438db54c6923ef23cf4a?tpId=137&&tqId=34095&rp=1&ru=/ta/exam-bytedance&qru=/ta/exam-bytedance/question-ranking
# 给定整数m以及n各数字A1,A2,..An，将数列A中所有元素两两异或，共能得到n(n-1)/2个结果，请求出这些结果中大于m的有多少个。
# 按照每个数的bit做trie，然后每次把当前数和trie一层层做异或，如果得到的数大于阈值，那么接下来的都满足
def xorLarger(arr,m):
    trie = dict()
    end = '#'
    t = max(arr+[m])
    highest = 0
    while t>0:
        highest+=1
        t = t>>1
    for val in arr:
        digit = 1<<highest
        cur = trie
        while digit!=0:
            t = val&digit
            if t not in cur:
                cur[t] = dict()
            cur = cur[t]
            digit = digit>>1
        cur[end] = cur.get(end,0)+1
    res = [0]

    def getCount(cur):#数一下当前trie里有多少个元素
        if end in cur:
            return cur[end]
        t = 0
        for key in cur:
            t+=getCount(cur[key])
        return t

    def dfs(val,cur,digit,curres):#对于val值，与cur进行处理，digit位，之前结果为curres
        if end in cur:
            return
        for key in cur:
            t = (val&digit)^key
            if curres+t>m:
                res[0]+=getCount(cur[key])
                return
            else:
                dfs(val,cur[key],digit>>1,curres+t)

    for val in arr:
        dfs(val,trie,1<<highest,0)
    return res[0]//2
xorLarger(arr = [1,2,3,5,7,4,2,1,9,4,2,1,5,8,6,4,2,5,9,4,2,1,5,7,6],m = 10)

47